In [1]:
import ee, geemap, pandas as pd
from datetime import datetime

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# 분석 대상 지역 좌표
lat, lon = 28.565038182965658, -81.15889424232911   # Florida, USA Beekeeper
point = ee.Geometry.Point(lon, lat).buffer(5000)

# 분석 기간
start_year, end_year = 2014, 2024
start_date = f"{start_year}-01-01"
end_date = f"{end_year}-12-31"


In [2]:
def ee_fc_to_pandas(fc):
    """GEE FeatureCollection → pandas DataFrame 변환"""
    data = fc.aggregate_array("EVI").getInfo()
    dates = fc.aggregate_array("date").getInfo()
    return pd.DataFrame({"EVI": data, "date": pd.to_datetime(dates)})
 
def get_modis_evi(point, start_date, end_date):
    modis = (
        ee.ImageCollection("MODIS/006/MOD13Q1")
        .filterDate(start_date, end_date)
        .select("EVI")
        .map(lambda img: img.multiply(0.0001).copyProperties(img, ["system:time_start"]))
    )

    def extract(img):
        mean = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=250,
            bestEffort=True
        )
        return ee.Feature(None, {
            "EVI": mean.get("EVI"),
            "date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")
        })

    evi_fc = modis.map(extract)
    evi_df = ee_fc_to_pandas(evi_fc)

    # 후처리
    evi_df["date"] = pd.to_datetime(evi_df["date"])
    evi_df["DOY"] = evi_df["date"].dt.dayofyear
    evi_df = evi_df.sort_values("date").reset_index(drop=True)
    return evi_df

evi_df = get_modis_evi(point, start_date, end_date)
print(evi_df.head())

c:\sehwi\NASAHackerton\.venv\Lib\site-packages\ee\deprecation.py:209: DeprecationWarning: 

Attention required for MODIS/006/MOD13Q1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD13Q1

  warnings.warn(warning, category=DeprecationWarning)


        EVI       date  DOY
0  0.338458 2014-01-01    1
1  0.297923 2014-01-17   17
2  0.263740 2014-02-02   33
3  0.282372 2014-02-18   49
4  0.330245 2014-03-06   65


In [3]:
def ee_fc_to_pandas_safe(fc, keys):
    arrays = {k: fc.aggregate_array(k).getInfo() for k in keys}
    min_len = min(len(v) for v in arrays.values())
    arrays = {k: v[:min_len] for k, v in arrays.items()}
    return pd.DataFrame(arrays)


def get_daymet_temp(point, start_date, end_date):
    daymet = (
        ee.ImageCollection("NASA/ORNL/DAYMET_V4")
        .filterDate(start_date, end_date)
        .select(["tmax", "tmin", "dayl", "prcp", "vp"])
    )

    def extract(img):
        mean = img.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=point,
            scale=1000,
            bestEffort=True
        )
        # tmax = ee.Number(mean.get("tmax"))
        tmax = ee.Algorithms.If(mean.contains("tmax"), mean.get("tmax"), -999)
        tmin = ee.Algorithms.If(mean.contains("tmin"), mean.get("tmin"), -999)
        dayl = ee.Algorithms.If(mean.contains("dayl"), mean.get("dayl"), -999)
        prcp = ee.Algorithms.If(mean.contains("prcp"), mean.get("prcp"), -999)
        vp = ee.Algorithms.If(mean.contains("vp"), mean.get("vp"), -999)
        return ee.Feature(None, {
            "tmax": tmax,
            "tmin": tmin,
            "dayl": dayl,
            "prcp": prcp,
            "vp": vp,
            "date": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd")
        })

    temp_fc = daymet.map(extract)
    met_df = ee_fc_to_pandas_safe(temp_fc, ["tmax", "tmin", "dayl", "prcp", "vp", "date"])
    met_df["date"] = pd.to_datetime(met_df["date"])
    met_df["DOY"] = met_df["date"].dt.dayofyear
    met_df = met_df.sort_values("date").reset_index(drop=True)
    return met_df

met_df = get_daymet_temp(point, start_date, end_date)
print(met_df.head())


        tmax       tmin          dayl      prcp           vp       date  DOY
0  19.673153  14.731668  36784.094970  2.156205  1675.245357 2014-01-01    1
1  26.422009  15.633673  36809.778666  8.354220  1775.294028 2014-01-02    2
2  18.108784   7.232691  36837.573028  0.000000  1017.511743 2014-01-03    3
3  19.607913   7.262861  36867.466580  0.000000  1019.680003 2014-01-04    4
4  25.794135  11.534537  36899.431665  0.000000  1359.405605 2014-01-05    5


In [4]:
outdir = r"C:\sehwi\NASAHackerton\data\output"
import os
os.makedirs(outdir, exist_ok=True)

evi_path = f"{outdir}/evi_df_{start_year}_{end_year}.csv"
met_path = f"{outdir}/met_df_{start_year}_{end_year}.csv"

evi_df.to_csv(evi_path, index=False)
met_df.to_csv(met_path, index=False)

print(f"EVI saved to: {evi_path}")
print(f"Met saved to: {met_path}")


EVI saved to: C:\sehwi\NASAHackerton\data\output/evi_df_2014_2024.csv
Met saved to: C:\sehwi\NASAHackerton\data\output/met_df_2014_2024.csv
